In [2]:
# from codebase.mgp import MovieGroupProcess
from codebase.utils import MongoConnector #, DocumentStream
from gensim.corpora import Dictionary
from datetime import datetime

## Step1, build Dictionary object

In [3]:
conn = MongoConnector("./config.ini")
conn.get_collection_cursor("FirstWeek_March")

In [14]:
%%time
query = {"tokens":{"$exists": True}}
dct = Dictionary()
for doc in conn.data_streaming_from_collection(query=query):
    token_f = [x for x in doc["tokens"] if len(x) > 1]
    dct.add_documents([token_f])

CPU times: user 43.8 s, sys: 372 ms, total: 44.2 s
Wall time: 52.2 s


In [15]:
print("Original size of vocabs: {}".format(len(dct)))
# control the vocabulary
dct.filter_extremes(no_below=20, no_above=0.5, keep_n=len(dct), keep_tokens=None)
print("Truncated size of vocabs: {}".format(len(dct)))

Original size of vocabs: 335223
Truncated size of vocabs: 29667


## Step2, export corpus

In [16]:
%%time

file_prefix = "./corpora/FirstWeek-March-Tweets"
meta_wf = open(file_prefix+"-Meta.csv", "w")
token_wf = open(file_prefix+".txt", "w")

query = {"tokens":{"$exists": True}}
for i, doc in enumerate(conn.data_streaming_from_collection(query=query)):
    
    if i % 100000 == 0:
        print("Have processed {} documents".format(i))
    
    trucated_tokens = [x for x in doc["tokens"] if x in dct.token2id.keys()]
    
    if len(trucated_tokens) > 3:
        timestamp = doc["created_at"].strftime("%Y-%m-%d %H:%M:%S")
        meta_wf.write("{},{}\n".format(doc["id_str"],timestamp))
        token_wf.write("{}\n".format(" ".join(trucated_tokens)))

meta_wf.close()
token_wf.close()

with open(file_prefix+"-Vocabs.txt", "w") as wf:
    for word in dct.token2id.keys():
        wf.write("{}\n".format(word))

Have processed 0 documents
Have processed 100000 documents
Have processed 200000 documents
Have processed 300000 documents
Have processed 400000 documents
Have processed 500000 documents
Have processed 600000 documents
Have processed 700000 documents
Have processed 800000 documents
Have processed 900000 documents
Have processed 1000000 documents
Have processed 1100000 documents
Have processed 1200000 documents
Have processed 1300000 documents
Have processed 1400000 documents
CPU times: user 22.4 s, sys: 540 ms, total: 22.9 s
Wall time: 31.2 s
